In [50]:
import pyspark
import pandas as pd
import numpy as np

from pyspark.sql.types import *
from pyspark.sql.functions import *


from pyspark import SparkContext
from pyspark.sql import SQLContext

from pyspark.sql import functions as F

from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import *
from pyspark import StorageLevel
from pyspark.sql.functions import udf

from pyspark.ml.feature import StopWordsRemover

import copy

from pyspark.sql.functions import pandas_udf

import textdistance

from tqdm import tqdm

#from past.builtins import xrange

from pyspark.sql.types import *

import pyspark.sql.types as T

import unidecode
import re
import csv
import datetime
from nltk.tokenize import word_tokenize

from elasticsearch import Elasticsearch

import nltk
from function2 import *

In [2]:
import pandas as pd
df1 = pd.read_csv('DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv', sep = ';')
df2 = pd.read_csv('DATA_RELACION_ENTIDADES_MAESTRO_short1.csv', sep = ';')

In [3]:
df1.columns

Index(['Id', 'Entidad', 'Origen_Solicitud', 'Identificadores_Origen',
       'Entidad_Norm', 'CIF', 'CIF_validacion', 'Centro', 'Centro_Norm',
       'Tipo', 'Provincia_Entidad', 'Pais_Entidad', 'Provincia_Centro', 'PIC',
       'Tipo_Persona', 'Entidad_Match', 'Provincia_Match', 'Match'],
      dtype='object')

In [4]:
df2.columns

Index(['Id_NIVEL_1', 'Id_NIVEL_0', 'ID_ENTIDAD_NIVEL_0', 'ID_ENTIDAD_NIVEL_1',
       'Jerarquia', 'NIF_COD_NIVEL_0', 'ACRONIMO_NIVEL_0',
       'NOMBRE_ENTIDAD_NIVEL_0', 'Nombre_Entidad_Mostrar_NIVEL_0',
       'TIPO_ENTIDAD_N1_1_NIVEL_0', 'TIPO_ENTIDAD_N2_1_NIVEL_0',
       'DIRECCION_POSTAL_NIVEL_0', 'COD_POSTAL_NIVEL_0',
       'COD_PROVINCIA_NIVEL_0', 'PROVINCIA_NIVEL_0', 'COD_CCAA_NIVEL_0',
       'CCAA_NIVEL_0', 'ENLACE_WEB_NIVEL_0', 'SOMMA_NIVEL_0',
       'TIPO_ENTIDAD_REGIONAL_NIVEL_0', 'ESTADO_x_NIVEL_0', 'NIF_COD_NIVEL_1',
       'ACRONIMO_NIVEL_1', 'NOMBRE_ENTIDAD_NIVEL_1',
       'Nombre_Entidad_Mostrar_NIVEL_1', 'TIPO_ENTIDAD_N1_1_NIVEL_1',
       'TIPO_ENTIDAD_N2_1_NIVEL_1', 'DIRECCION_POSTAL_NIVEL_1',
       'COD_POSTAL_NIVEL_1', 'COD_PROVINCIA_NIVEL_1', 'PROVINCIA_NIVEL_1',
       'COD_CCAA_NIVEL_1', 'CCAA_NIVEL_1', 'ENLACE_WEB_NIVEL_1',
       'SOMMA_NIVEL_1', 'TIPO_ENTIDAD_REGIONAL_NIVEL_1', 'ESTADO_x_NIVEL_1'],
      dtype='object')

In [5]:
df1columns = ['Entidad', 'Origen_Solicitud']
df2columns = ['ID_ENTIDAD_NIVEL_0', 'NOMBRE_ENTIDAD_NIVEL_0']

In [5]:
columnsSourcesNames = [[]]*len(df1columns)
for k in range(len(df1columns)):
    columnsSourcesNames[k] = list(df1[df1columns[k]])

In [7]:
columnsSourcesNames = list(map(lambda x: list(df1[df1columns[x]]), range(len(df1columns))))

In [8]:
columnstargetNames = list(map(lambda x: list(df2[df2columns[x]]), range(len(df2columns))))

In [9]:
es = Elasticsearch('http://localhost:9200/')
    
for i in list(es.indices.get_alias().items()):
    es.indices.delete(index=list(i)[0])

In [10]:
es.indices.delete(index='test-index', ignore=[400, 404])

/tmp/ipykernel_22922/3501884637.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='test-index', ignore=[400, 404])


ObjectApiResponse({'error': {'root_cause': [{'type': 'index_not_found_exception', 'reason': 'no such index [test-index]', 'resource.type': 'index_or_alias', 'resource.id': 'test-index', 'index_uuid': '_na_', 'index': 'test-index'}], 'type': 'index_not_found_exception', 'reason': 'no such index [test-index]', 'resource.type': 'index_or_alias', 'resource.id': 'test-index', 'index_uuid': '_na_', 'index': 'test-index'}, 'status': 404})

In [11]:
index_name = 'index_name'

In [12]:
for name in tqdm(columnsSourcesNames[0]):
    i= columnsSourcesNames[0].index(name)
    doc = {'entity':name}
    es.index(index=index_name, id=i, document=doc)

100%|█████████████████████████████████████████| 970/970 [01:01<00:00, 15.80it/s]


In [38]:
[es.index(index=index_name, id=columnsSourcesNames[0].index(name), document={'entity':name}) for name in tqdm(columnsSourcesNames[0])]

100%|█████████████████████████████████████████| 970/970 [01:31<00:00, 10.59it/s]


[ObjectApiResponse({'_index': 'index_name', '_id': '0', '_version': 86, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1216, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '0', '_version': 87, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1217, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '2', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1218, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '3', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1219, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '4', '_version': 8, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1220, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '5', '_version': 

In [39]:
list(map(lambda x: es.index(index=index_name, id=columnsSourcesNames[0].index(x), document={'entity':x}), tqdm(columnsSourcesNames[0][:1000])))

100%|█████████████████████████████████████████| 970/970 [01:08<00:00, 14.09it/s]


[ObjectApiResponse({'_index': 'index_name', '_id': '0', '_version': 152, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2186, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '0', '_version': 153, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2187, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '2', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2188, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '3', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2189, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '4', '_version': 12, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2190, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'index_name', '_id': '5', '_version

In [64]:
source_name = list(df1['Entidad'])

In [66]:
matching_entities = []
query = {"from": 0,
         "size": 10,
         "query": {"match": {"entity":{'query':source_name[0],'fuzziness':'AUTO'}}},
         "sort":[{'_score':{'order':'desc'}}]}
resp = es.search(index=index_name, body=query)

/tmp/ipykernel_22922/576026093.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.search(index=index_name, body=query)


In [ ]:
resp['hits']['hits']

[{'_index': 'index_name',
  '_id': '0',
  '_score': 24.298605,
  '_source': {'entity': 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)'}},
 {'_index': 'index_name',
  '_id': '320',
  '_score': 23.951323,
  '_source': {'entity': 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS (CSIC)'}},
 {'_index': 'index_name',
  '_id': '20',
  '_score': 21.719637,
  '_source': {'entity': 'Agencia Estatal Consejo Superior de Investigaciones Científicas'}},
 {'_index': 'index_name',
  '_id': '454',
  '_score': 16.153856,
  '_source': {'entity': 'CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS'}},
 {'_index': 'index_name',
  '_id': '123',
  '_score': 6.499949,
  '_source': {'entity': 'CSIC'}},
 {'_index': 'index_name',
  '_id': '92',
  '_score': 4.3581376,
  '_source': {'entity': 'CENTRO DE INVESTIGACION COOP DE ENERGIAS ALTERNATIVAS CIC ENERGIGUNE'}},
 {'_index': 'index_name',
  '_id': '389',
  '_score': 4.2075996,
  '_source': {'entity': 'ASOCIACION CENTRO DE IN

In [ ]:
element = resp['hits']['hits'][0]
element

{'_index': 'index_name',
 '_id': '0',
 '_score': 24.298605,
 '_source': {'entity': 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)'}}

In [133]:
results = {}
source_idx=0
matching_entities = []
for element in resp['hits']['hits']:
    target_idx = int(element['_id'])
    entity_j = element['_source']['entity']

    firstMatch = [target_idx,entity_j]

    secondMatch = [columnstargetNames[l][target_idx] for l in range(len(df2columns))]
        
    match = tuple(firstMatch + secondMatch)
    matching_entities.append(match)
    
dicCol = dict([(df1columns[l], columnsSourcesNames[l][source_idx]) for l in range(len(df1columns))])
results[source_idx] = {'source_name':source_name[0], 'matching_names_by_elasticsearch':matching_entities}
results[source_idx].update(dicCol)

In [148]:
results[0].keys()

dict_keys(['source_name', 'matching_names_by_elasticsearch', 'Entidad', 'Origen_Solicitud'])

In [157]:
columnsSourcesNames1

[['AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)'],
 ['ACT_NAC',
  'ACT_NAC',
  'ACT_NAC',
  'ACT_NAC',
  'ACT_NAC',
  'ACT_NAC',
  'ACT_NAC',
  'ACT_NAC',
  'ACT_NAC',
  'ACT_NAC']]

In [154]:
columnsSourcesNames1 = list(map(lambda k: [columnsSourcesNames1[k]] + [columnsSourcesNamesResults[k]], range(len(df1columns))))
columnsSourcesNames1


[['AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)'],
 ['ACT_NAC', 'ACT_NAC']]

In [166]:
columnsTargetNames1


[[157, 757, 515, 853, 100013, 947, 239, 841, 297, 896],
 ['INSTITUTO NACIONAL DE BIOINFORMATICA INB',
  'INSTITUT D´INVESTIGACIO BIOMEDICA DE GIRONA DR.JOSEP TRUETA',
  'INSTITUT D´INVESTIGACIO SANITARI PERE VIRGILI',
  'Universidad de Extremadura',
  'None',
  'Red Académica y de Investigación Española (RedIRIS)',
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICA, M.P. CSIC',
  'Universidad de Alicante',
  'CIBER BIOINGENIERIA BIOMATERIALES Y NANOMEDICINA CIBER BBN',
  'Universidad de Deusto']]

In [168]:
names_source

['AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)']

In [169]:
names_target

['AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS (CSIC)',
 'Agencia Estatal Consejo Superior de Investigaciones Científicas',
 'CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS',
 'CSIC',
 'CENTRO DE INVESTIGACION COOP DE ENERGIAS ALTERNATIVAS CIC ENERGIGUNE',
 'ASOCIACION CENTRO DE INVESTIGACION COOP EN BIOCIENCIAS CIC BIOGUNE',
 'UNIVERSIDAD DE VALENCIA',
 'UNIVERSIDAD POLITÉCNICA DE VALENCIA',
 'Agencia Pública Empresarial Sanitaria Costa del Sol']

In [196]:
names_source

['AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)']

In [172]:
results[i]['matching_names_by_elasticsearch'][2]

(20,
 'Agencia Estatal Consejo Superior de Investigaciones Científicas',
 515,
 'INSTITUT D´INVESTIGACIO SANITARI PERE VIRGILI')

In [190]:
results[0]['matching_names_by_elasticsearch']

[(0,
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  157,
  'INSTITUTO NACIONAL DE BIOINFORMATICA INB'),
 (320,
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS (CSIC)',
  757,
  'INSTITUT D´INVESTIGACIO BIOMEDICA DE GIRONA DR.JOSEP TRUETA'),
 (20,
  'Agencia Estatal Consejo Superior de Investigaciones Científicas',
  515,
  'INSTITUT D´INVESTIGACIO SANITARI PERE VIRGILI'),
 (454,
  'CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS',
  853,
  'Universidad de Extremadura'),
 (123, 'CSIC', 100013, 'None'),
 (92,
  'CENTRO DE INVESTIGACION COOP DE ENERGIAS ALTERNATIVAS CIC ENERGIGUNE',
  947,
  'Red Académica y de Investigación Española (RedIRIS)'),
 (389,
  'ASOCIACION CENTRO DE INVESTIGACION COOP EN BIOCIENCIAS CIC BIOGUNE',
  239,
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICA, M.P. CSIC'),
 (135, 'UNIVERSIDAD DE VALENCIA', 841, 'Universidad de Alicante'),
 (471,
  'UNIVERSIDAD POLITÉCNICA DE VALENCIA',
  297,
  'CIBER B

In [193]:
names_target = list(map(lambda x: x[1], results[i]['matching_names_by_elasticsearch']))
names_source = list(map(lambda x: results[x]['source_name'], results)) * len(names_target)

In [194]:
names_target

['AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS (CSIC)',
 'Agencia Estatal Consejo Superior de Investigaciones Científicas',
 'CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS',
 'CSIC',
 'CENTRO DE INVESTIGACION COOP DE ENERGIAS ALTERNATIVAS CIC ENERGIGUNE',
 'ASOCIACION CENTRO DE INVESTIGACION COOP EN BIOCIENCIAS CIC BIOGUNE',
 'UNIVERSIDAD DE VALENCIA',
 'UNIVERSIDAD POLITÉCNICA DE VALENCIA',
 'Agencia Pública Empresarial Sanitaria Costa del Sol']

In [200]:
columnsSourcesNames1 = [[]]*len(df1columns)
columnsTargetNames1 = [[]]*len(df2columns)
for i in results:
    
    source_name = results[i]['source_name']
    columnsSourcesNamesResults = [None]*len(df1columns)
    
    columnsSourcesNamesResults = list(map(lambda x: results[i][df1columns[x]], range(len(df1columns))))
    # for l in range(len(df1columns)):
    #     columnsSourcesNamesResults[l] = results[i][df1columns[l]]
    names_target = list(map(lambda x: x[1], results[i]['matching_names_by_elasticsearch']))
    names_source = list(map(lambda x: results[x]['source_name'], results)) * len(names_target)
    
    for element in results[i]['matching_names_by_elasticsearch']:
        columnsSourcesNames1 = list(map(lambda k: columnsSourcesNames1[k] + [columnsSourcesNamesResults[k]], range(len(df1columns))))
        columnsTargetNamesResults = list(map(lambda x: element[x+2], range(len(df2columns))))
        columnsTargetNames1 = list(map(lambda x: columnsTargetNames1[x] + [columnsTargetNamesResults[x]], range(len(df2columns))))    

dicCol = dict([(df1columns[l], columnsSourcesNames1[l]) for l in range(len(df1columns))])
# dicCol = {}
# for n in range(len(df1columns)):
#     dicCol[df1columns[n]] = columnsSourcesNames1[n]

data = {'source_name':names_source, 'target_name':names_target}

data.update(dicCol)

dicCol = dict([(df2columns[l], columnsTargetNames1[l]) for l in range(len(df2columns))])
# dicCol = {}
# for n in range(len(df2columns)):
#     dicCol[df2columns[n]] = columnsTargetNames1[n]
data.update(dicCol)

result = pd.DataFrame(data)

In [204]:
list(data.keys())

['source_name',
 'target_name',
 'Entidad',
 'Origen_Solicitud',
 'ID_ENTIDAD_NIVEL_0',
 'NOMBRE_ENTIDAD_NIVEL_0']

22/08/21 15:20:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 150317 ms exceeds timeout 120000 ms
22/08/21 15:20:26 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1053)
	at org.apache.spark.e

In [201]:
result

,source_name,target_name,Entidad,Origen_Solicitud,ID_ENTIDAD_NIVEL_0,NOMBRE_ENTIDAD_NIVEL_0
0,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,157,INSTITUTO NACIONAL DE BIOINFORMATICA INB
1,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,757,INSTITUT D´INVESTIGACIO BIOMEDICA DE GIRONA DR...
2,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,Agencia Estatal Consejo Superior de Investigac...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,515,INSTITUT D´INVESTIGACIO SANITARI PERE VIRGILI
3,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,853,Universidad de Extremadura
4,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,CSIC,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,100013,None
5,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,CENTRO DE INVESTIGACION COOP DE ENERGIAS ALTER...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,947,Red Académica y de Investigación Española (Red...
6,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ASOCIACION CENTRO DE INVESTIGACION COOP EN BIO...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,239,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...
7,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,UNIVERSIDAD DE VALENCIA,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,841,Universidad de Alicante
8,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,UNIVERSIDAD POLITÉCNICA DE VALENCIA,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,297,CIBER BIOINGENIERIA BIOMATERIALES Y NANOMEDICI...
9,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,Agencia Pública Empresarial Sanitaria Costa de...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,896,Universidad de Deusto


In [167]:
result

,source_name,target_name,Entidad,Origen_Solicitud,ID_ENTIDAD_NIVEL_0,NOMBRE_ENTIDAD_NIVEL_0
0,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,157,INSTITUTO NACIONAL DE BIOINFORMATICA INB
1,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,757,INSTITUT D´INVESTIGACIO BIOMEDICA DE GIRONA DR...
2,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,Agencia Estatal Consejo Superior de Investigac...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,515,INSTITUT D´INVESTIGACIO SANITARI PERE VIRGILI
3,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,853,Universidad de Extremadura
4,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,CSIC,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,100013,None
5,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,CENTRO DE INVESTIGACION COOP DE ENERGIAS ALTER...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,947,Red Académica y de Investigación Española (Red...
6,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ASOCIACION CENTRO DE INVESTIGACION COOP EN BIO...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,239,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...
7,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,UNIVERSIDAD DE VALENCIA,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,841,Universidad de Alicante
8,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,UNIVERSIDAD POLITÉCNICA DE VALENCIA,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,297,CIBER BIOINGENIERIA BIOMATERIALES Y NANOMEDICI...
9,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,Agencia Pública Empresarial Sanitaria Costa de...,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,ACT_NAC,896,Universidad de Deusto


In [105]:
secondMatch1 = [columnstargetNames[l][target_idx] for l in range(len(df2columns))]

In [117]:
secondMatch1

[157, 'INSTITUTO NACIONAL DE BIOINFORMATICA INB']

In [108]:
firstMatch

[0, 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)']

In [130]:
for source_idx in range(10):
    secondMatch1 = [columnstargetNames[l][int(x['_id'])] for l,x in zip(range(len(df2columns)),resp['hits']['hits'])]
    matching_entities = list(map(lambda x: tuple([int(x['_id']), x['_source']['entity']]+ secondMatch1), resp['hits']['hits']))

In [116]:
matching_entities

[(0,
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
  [896]),
 (320,
  'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTÍFICAS (CSIC)',
  ['Universidad de Deusto'])]

In [109]:
match = tuple(firstMatch + secondMatch)
match

(0,
 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 157,
 'INSTITUTO NACIONAL DE BIOINFORMATICA INB')

In [ ]:
match = tuple()

In [ ]:
secondMatch

[157, 'INSTITUTO NACIONAL DE BIOINFORMATICA INB']

In [77]:
dicCol = {}

for l in range(len(df1columns)):
    dicCol[df1columns[l]] = columnsSourcesNames[l][source_idx]


In [99]:
dicCol1 = {}
dicCol1.keys() = [df1columns[l] for l in range(len(df1columns))]
dicCol1.values() = [columnsSourcesNames[l][source_idx] for l in range(len(df1columns))]

SyntaxError: cannot assign to function call (3717551075.py, line 2)

In [103]:
dicCol1 = dict([(df1columns[l], columnsSourcesNames[l][source_idx]) for l in range(len(df1columns))])

In [104]:
dicCol1

{'Entidad': 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'Origen_Solicitud': 'ACT_NAC'}

In [81]:
map(lambda x: dicCol[df1columns[l]] = columnsSourcesNames[l][source_idx])

'ACT_NAC'

In [84]:
dicCol1 = dict(map(lambda x: df1columns[l]+ ':'+ columnsSourcesNames[x][source_idx], range(len(df1columns)))) 

ValueError: dictionary update sequence element #0 has length 87; 2 is required

In [78]:
dicCol

{'Entidad': 'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)',
 'Origen_Solicitud': 'ACT_NAC'}

In [ ]:
def format_results(results,df1columns,df2columns):
    names_source = []
    names_target = []
    columnsSourcesNames1 = [[]]*len(df1columns)
    columnsTargetNames1 = [[]]*len(df2columns)
    
    i=0
    for i in results:
        
        source_name = results[i]['source_name']
        columnsSourcesNamesResults = [None]*len(df1columns)
        
        for l in range(len(df1columns)):
            columnsSourcesNamesResults[l] = results[i][df1columns[l]]

        for element in results[i]['matching_names_by_elasticsearch']:
            target_name = element[1]
            names_source.append(source_name)
            names_target.append(target_name)
            

            for k in range(len(df1columns)):
                columnsSourcesNames1[k] = columnsSourcesNames1[k] + [columnsSourcesNamesResults[k]] 

            columnsTargetNamesResults = [None]*len(df2columns)
            
            for m in range(len(df2columns)):
                columnsTargetNamesResults[m] = element[m+2]
                
            for j in range(len(df2columns)):
                columnsTargetNames1[j] = columnsTargetNames1[j] + [columnsTargetNamesResults[j]] 
                
    dicCol = {}
    for n in range(len(df1columns)):
        dicCol[df1columns[n]] = columnsSourcesNames1[n]
    
    data = {'source_name':names_source, 'target_name':names_target}
    
    data.update(dicCol)
    
    dicCol = {}
    for n in range(len(df2columns)):
        dicCol[df2columns[n]] = columnsTargetNames1[n]
    
    data.update(dicCol)
    
    result = pd.DataFrame(data)
            
    return result

In [54]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

22/08/21 09:30:16 WARN Utils: Your hostname, gaszsantana resolves to a loopback address: 127.0.1.1; using 192.168.8.102 instead (on interface wlp1s0)
22/08/21 09:30:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/21 09:30:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [62]:
import requests;
res = requests.get('http://localhost:9200');
print(res.content)

b'{\n  "name" : "gaszsantana",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "tZRtuTGvQ8amEWllkc11zg",\n  "version" : {\n    "number" : "8.3.2",\n    "build_type" : "tar",\n    "build_hash" : "8b0b1f23fbebecc3c88e4464319dea8989f374fd",\n    "build_date" : "2022-07-06T15:15:15.901688194Z",\n    "build_snapshot" : false,\n    "lucene_version" : "9.2.0",\n    "minimum_wire_compatibility_version" : "7.17.0",\n    "minimum_index_compatibility_version" : "7.0.0"\n  },\n  "tagline" : "You Know, for Search"\n}\n'
